# CV Lab 8
# Shresht Mishra
# 211020450

In [14]:
import cv2
import numpy as np
import subprocess

## 1.Object detection from a given video.

In [2]:
yolo_net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [3]:
with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")

In [4]:
video_path = "D:/CV/Lab 8/title-artist.mp4"
cap = cv2.VideoCapture(video_path)

In [5]:
output_path = "D:/CV/Lab 8/output_video_yolo.mp4"
codec = cv2.VideoWriter_fourcc(*"XVID")
output_writer = cv2.VideoWriter(output_path, codec, 30.0, (int(cap.get(3)), int(cap.get(4))))

In [8]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get the blob from the frame and perform YOLO object detection
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    yolo_net.setInput(blob)
    layer_names = yolo_net.getUnconnectedOutLayersNames()
    detections = yolo_net.forward(layer_names)

    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                # Get the coordinates and size of the bounding box
                center_x, center_y, width, height = map(int, obj[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]))
                x, y = center_x - width // 2, center_y - height // 2

                # Draw bounding box and label on the frame
                color = (0, 255, 0)  # Green
                cv2.rectangle(frame, (x, y), (x + width, y + height), color, 2)
                label = f"{classes[class_id]}: {confidence:.2f}"
                cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the frame with bounding boxes to the output video
    output_writer.write(frame)
    
cap.release()
output_writer.release()
cv2.destroyAllWindows()

## 2.Count the object from given images

In [5]:
classes_to_count = ["car", "motorbike", "person"]

In [6]:
class_counts = {cls: 0 for cls in classes_to_count}

In [10]:
object_tracker = {}
object_id_counter = 1

In [11]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get the blob from the frame and perform YOLO object detection
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    yolo_net.setInput(blob)
    layer_names = yolo_net.getUnconnectedOutLayersNames()
    detections = yolo_net.forward(layer_names)

    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                detected_class = classes[class_id]
                if detected_class in classes_to_count:
                    # Get the coordinates and size of the bounding box
                    center_x, center_y, width, height = map(int, obj[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]))
                    x, y = center_x - width // 2, center_y - height // 2

                    # Check if the object is already being tracked
                    object_id = None
                    for obj_id, (prev_x, prev_y, _, _) in object_tracker.items():
                        if x >= prev_x and x <= prev_x + width and y >= prev_y and y <= prev_y + height:
                            object_id = obj_id
                            break

                    if object_id is None:
                        # New object detected, assign a unique ID
                        object_id = object_id_counter
                        object_id_counter += 1

                    # Update the object tracker with the new coordinates
                    object_tracker[object_id] = (x, y, width, height)

                    # Increment the count for the detected class
                    class_counts[detected_class] += 1

    # Draw counts on the frame
    for cls, count in class_counts.items():
        text = f"{cls}: {count}"
        cv2.putText(frame, text, (10, 30 * (1 + classes_to_count.index(cls))), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame with counts
    cv2.imshow("Object Counting", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [12]:
for cls, count in class_counts.items():
    print(f"Total {cls}s detected: {count}")

Total cars detected: 11539
Total motorbikes detected: 2903
Total persons detected: 7629


## 3.Apply compression and decompression concept.

In [18]:
output_video_path = "D:/CV/Lab 8/compressed_video.mp4"

In [22]:
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frames_per_second = int(cap.get(5))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change the codec as needed
out = cv2.VideoWriter(output_video_path, fourcc, frames_per_second, (frame_width, frame_height))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform any desired processing on the frame (e.g., object detection, etc.)
    # Compressing here can also be done, but typically compression is done during export

    # Write the processed frame to the output video
    out.write(frame)

# Release the video objects
cap.release()
out.release()
cv2.destroyAllWindows()

In [23]:
input_compressed_video_path = "D:/CV/Lab 8/compressed_video.mp4"
output_decompressed_video_path = "D:/CV/Lab 8/decompressed_video.mp4"

In [24]:
cap = cv2.VideoCapture(input_compressed_video_path)

In [25]:
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frames_per_second = int(cap.get(5))

# Define the codec and create a VideoWriter object for the decompressed video
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change the codec as needed
out = cv2.VideoWriter(output_decompressed_video_path, fourcc, frames_per_second, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform any desired processing on the frame if needed
    
    # Write the frame to the decompressed video
    out.write(frame)

# Release the video objects
cap.release()
out.release()
cv2.destroyAllWindows()